In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

import datetime

from sklearn.cluster import MiniBatchKMeans
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import joblib

In [34]:
# import dataset
df = pd.read_csv(r'yellow_tripdata_2015-01.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,N,-73.974785,40.750618,1,12.0,1.0,0.5,3.25,0.0,0.3,17.05
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,N,-73.994415,40.759109,1,14.5,0.5,0.5,2.00,0.0,0.3,17.80
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,N,-73.951820,40.824413,2,9.5,0.5,0.5,0.00,0.0,0.3,10.80
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,N,-74.004326,40.719986,2,3.5,0.5,0.5,0.00,0.0,0.3,4.80
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,N,-74.004181,40.742653,2,15.0,0.5,0.5,0.00,0.0,0.3,16.30


In [35]:
def sample_uniform_by_day(
    df: pd.DataFrame,
    datetime_col: str = "tpep_pickup_datetime",
    n_per_day: int | None = None,
    frac_per_day: float | None = None,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Samples uniformly at random within each day.
- Choose either `n_per_day` (fixed number of rows per day) or `frac_per_day` (fraction per day).
- If `n_per_day` > size of a given day, take all rows from that day.
    """
    if (n_per_day is None) == (frac_per_day is None):
        raise ValueError("Spécifie exactement l'un de `n_per_day` ou `frac_per_day`.")

    # Make sure the time column is set to datetime
    df = df.copy()
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors="coerce")
    df = df.dropna(subset=[datetime_col])

    # Key ‘date’ (calendar day) from the time of pick-up
    df["_date"] = df[datetime_col].dt.date

    if n_per_day is not None:
        # Sampling with a fixed number per day (without replacement)
        out = (
            df.groupby("_date", group_keys=False)
              .apply(lambda g: g.sample(n=min(n_per_day, len(g)),
                                        replace=False,
                                        random_state=random_state))
        )
    else:
         # Sampling with one fraction per day
        out = (
            df.groupby("_date", group_keys=False)
              .apply(lambda g: g.sample(frac=frac_per_day,
                                        replace=False,
                                        random_state=random_state))
        )

    return out.drop(columns=["_date"])

In [36]:
#df = sample_uniform_by_day(df, frac_per_day=0.1)

In [37]:
# Quick overview of distribution of the numerical values
df.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
count,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274899e+07,1.274898e+07,1.274899e+07
mean,1.521437e+00,1.681491e+00,1.345913e+01,-7.256184e+01,3.997282e+01,1.036901e+00,-7.260904e+01,3.999961e+01,1.386712e+00,1.190566e+01,3.082790e-01,4.977986e-01,1.853814e+00,2.434984e-01,2.831431e-01,1.510829e+01
std,4.995402e-01,1.337924e+00,9.844094e+03,1.012510e+01,5.578691e+00,6.732240e-01,9.966037e+00,5.487742e+00,4.988611e-01,1.030254e+01,5.916643e-01,3.534229e-02,1.106432e+03,1.527171e+00,6.908633e-02,1.106503e+03
min,1.000000e+00,0.000000e+00,0.000000e+00,-1.219258e+02,0.000000e+00,1.000000e+00,-7.401667e+02,-9.029157e+00,1.000000e+00,-4.500000e+02,-7.900000e+01,-5.000000e-01,-9.242000e+01,-2.600000e+01,0.000000e+00,-4.503000e+02
25%,1.000000e+00,1.000000e+00,1.000000e+00,-7.399168e+01,4.073554e+01,1.000000e+00,-7.399120e+01,4.073436e+01,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.160000e+00
50%,2.000000e+00,1.000000e+00,1.680000e+00,-7.398160e+01,4.075314e+01,1.000000e+00,-7.397976e+01,4.075362e+01,1.000000e+00,9.000000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01
75%,2.000000e+00,2.000000e+00,3.000000e+00,-7.396662e+01,4.076757e+01,1.000000e+00,-7.396246e+01,4.076880e+01,2.000000e+00,1.350000e+01,5.000000e-01,5.000000e-01,2.060000e+00,0.000000e+00,3.000000e-01,1.630000e+01
max,2.000000e+00,9.000000e+00,1.542000e+07,7.866265e+01,4.047000e+02,9.900000e+01,8.527402e+01,4.595333e+02,5.000000e+00,4.008000e+03,9.999900e+02,5.000000e-01,3.950589e+06,1.450090e+03,3.000000e-01,3.950612e+06


In [38]:
# Check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12748986 entries, 0 to 12748985
Data columns (total 19 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               int64  
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        int64  
 4   trip_distance          float64
 5   pickup_longitude       float64
 6   pickup_latitude        float64
 7   RateCodeID             int64  
 8   store_and_fwd_flag     object 
 9   dropoff_longitude      float64
 10  dropoff_latitude       float64
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  float64
 18  total_amount           float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.8+ GB


In [39]:
df = df.dropna()

In [40]:
df = df[df["trip_distance"] > 0]

def remove_outliers(df, column):
    Q1 = df[column].quantile(25/100)
    Q3 = df[column].quantile(75/100)
    IQR = Q3 - Q1
    
    lim_inf = Q1 - 1.5 * IQR
    lim_sup = Q3 + 1.5 * IQR
    
    df_clean = df[(df[column] >= lim_inf) & (df[column] <= lim_sup)]
    return df_clean

In [41]:
# One-hot encode 
df["store_and_fwd_flag"] = df["store_and_fwd_flag"].map(lambda x: 0 if x=='N' else 1)


In [42]:
# Convert time to timestamps
format_code = "%Y-%m-%d %H:%M:%S"

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], format=format_code)
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], format=format_code)

In [43]:
# Number of trips per date
df["pickup_date"] = df["tpep_pickup_datetime"].dt.date
trips_per_date = df.groupby("pickup_date").size()

In [44]:
# Make 'pickup_date' to a datetime object
df['pickup_date'] = pd.to_datetime(df['pickup_date'], format=format_code)

# Extract the name of the day
df['day_of_week'] = df['pickup_date'].dt.day_name()

# Define the correct order
ordered_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Convert to an ordered categorical
df["day_of_week"] = pd.Categorical(df["day_of_week"], categories=ordered_days, ordered=True)

# Compute average fare per day of week
avg_fare_amount_per_day_of_week = df.groupby('day_of_week')['fare_amount'].mean()


C:\Users\njee2\AppData\Local\Temp\ipykernel_16648\3366026511.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_fare_amount_per_day_of_week = df.groupby('day_of_week')['fare_amount'].mean()


In [45]:
# Categorize the different moments of a day (early morning, morning, noon/afternoon, night)

def map_moments_of_day(x):
    if x.hour in range(0,6):
        return "early morning"
    elif x.hour in range(6, 13):
        return 'morning'
    elif x.hour in range(13, 16):
        return 'noon/afternoon'
    elif x.hour in range(16,21):
        return 'rush hour'
    else:
        return "night"

In [46]:
# Create a column moments of day
df['Moments_of_day'] = df['tpep_pickup_datetime'].apply(map_moments_of_day)

In [47]:
# Create time duration feature
df['duration_in_min'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [48]:
df['duration_in_min'].describe()

count    1.266962e+07
mean     1.423901e+01
std      6.508883e+02
min     -1.211017e+03
25%      6.150000e+00
50%      1.000000e+01
75%      1.583333e+01
max      5.485556e+05
Name: duration_in_min, dtype: float64

In [49]:
# On garde uniquement les trajets entre 1 minute et 120 minutes (2 heures)
df = df[(df['duration_in_min'] > 1) & (df['duration_in_min'] < 120)]
df['duration_in_min'].describe()

count    1.261742e+07
mean     1.236981e+01
std      9.116321e+00
min      1.016667e+00
25%      6.183333e+00
50%      1.001667e+01
75%      1.583333e+01
max      1.199000e+02
Name: duration_in_min, dtype: float64

In [50]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_date,duration_in_min
count,1.261742e+07,12617420,12617420,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,...,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,1.261742e+07,12617420,1.261742e+07
mean,1.522028e+00,2015-01-16 14:48:31.681076480,2015-01-16 15:00:53.869871616,1.683486e+00,1.307477e+01,-7.266342e+01,4.002876e+01,1.027605e+00,8.830331e-03,-7.278807e+01,...,1.382509e+00,1.186211e+01,3.090090e-01,4.986623e-01,1.853225e+00,2.416275e-01,2.832245e-01,1.506404e+01,2015-01-16 00:43:44.960599552,1.236981e+01
min,1.000000e+00,2015-01-01 00:00:00,2015-01-01 00:01:25,0.000000e+00,1.000000e-02,-1.193147e+02,0.000000e+00,1.000000e+00,0.000000e+00,-7.401667e+02,...,1.000000e+00,-2.422500e+02,-7.900000e+01,-5.000000e-01,-9.242000e+01,-9.750000e+00,0.000000e+00,-2.425500e+02,2015-01-01 00:00:00,1.016667e+00
25%,1.000000e+00,2015-01-09 09:32:47,2015-01-09 09:49:31,1.000000e+00,1.000000e+00,-7.399171e+01,4.073574e+01,1.000000e+00,0.000000e+00,-7.399122e+01,...,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,3.000000e-01,8.300000e+00,2015-01-09 00:00:00,6.183333e+00
50%,2.000000e+00,2015-01-16 11:48:11,2015-01-16 12:01:53,1.000000e+00,1.700000e+00,-7.398165e+01,4.075323e+01,1.000000e+00,0.000000e+00,-7.397984e+01,...,1.000000e+00,9.000000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.116000e+01,2015-01-16 00:00:00,1.001667e+01
75%,2.000000e+00,2015-01-23 19:18:00,2015-01-23 19:31:07,2.000000e+00,3.020000e+00,-7.396680e+01,4.076762e+01,1.000000e+00,0.000000e+00,-7.396275e+01,...,2.000000e+00,1.350000e+01,5.000000e-01,5.000000e-01,2.080000e+00,0.000000e+00,3.000000e-01,1.630000e+01,2015-01-23 00:00:00,1.583333e+01
max,2.000000e+00,2015-01-31 23:59:59,2015-02-01 01:09:52,9.000000e+00,1.542000e+07,7.866265e+01,4.047000e+02,9.900000e+01,1.000000e+00,8.527402e+01,...,5.000000e+00,4.008000e+03,6.480200e+02,5.000000e-01,3.950589e+06,1.450090e+03,3.000000e-01,3.950612e+06,2015-01-31 00:00:00,1.199000e+02
std,4.995145e-01,NaN,NaN,1.339387e+00,9.854522e+03,9.761568e+00,5.378480e+00,5.179246e-01,9.355403e-02,9.302957e+00,...,4.946890e-01,9.772367e+00,5.057920e-01,2.739460e-02,1.112186e+03,1.456616e+00,6.892909e-02,1.112250e+03,NaN,9.116321e+00


In [51]:

# 0-5,5-10, 10-20, 20-40, 40-max
bins = [0,5 ,10, 20, 40, df['duration_in_min'].max()]
labels = ['Very Short (<5m)', 'Short (5-10m)', 'Medium (10-20m)', 'Long (20-40m)', 'Very Long (>40m)']

# Création de la nouvelle colonne 'Duration_Category'
df['Duration_Category'] = pd.cut(
    df['duration_in_min'], 
    bins=bins, 
    labels=labels, 
    include_lowest=True
)

# Vérification rapide
print("Répartition des types de trajets :")
print(df['Duration_Category'].value_counts().sort_index())

# Petit aperçu pour voir si ça semble cohérent avec le prix
print("\nPrix moyen par catégorie de durée :")
print(df.groupby('Duration_Category')['fare_amount'].mean())

Répartition des types de trajets :
Duration_Category
Very Short (<5m)    2115434
Short (5-10m)       4185229
Medium (10-20m)     4414131
Long (20-40m)       1687715
Very Long (>40m)     214911
Name: count, dtype: int64

Prix moyen par catégorie de durée :
Duration_Category
Very Short (<5m)     4.774898
Short (5-10m)        7.306917
Medium (10-20m)     12.555483
Long (20-40m)       25.780489
Very Long (>40m)    46.789196
Name: fare_amount, dtype: float64


C:\Users\njee2\AppData\Local\Temp\ipykernel_16648\3502994053.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('Duration_Category')['fare_amount'].mean())


In [52]:
# Create average speed (mph) feature
df["avg_speed"] = 60*(df["trip_distance"]/df["duration_in_min"]) #in mph

In [53]:
# Code to find zones based on longitude and latitude TODO



def find_clusters_kmeans(df, lon_col, lat_col, new_cluster_col, n_clusters=100):
    print(f"Clustering de '{new_cluster_col}' en {n_clusters} zones...")
    
    
    coords_df = df[[lon_col, lat_col]].dropna()
    coords_np = coords_df.values
    
    # Training Step
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, batch_size=2048, n_init='auto')
    kmeans.fit(coords_np)
    
    
    labels = kmeans.predict(coords_np)
    
    df_clustered = df.copy()
    df_clustered[new_cluster_col] = -1
    df_clustered.loc[coords_df.index, new_cluster_col] = labels
    df_clustered[new_cluster_col] = df_clustered[new_cluster_col].astype(int)
    

    return df_clustered, kmeans


# Clustering 
df, pickup_model = find_clusters_kmeans(
    df, 
    'pickup_longitude', 'pickup_latitude', 
    'Pickup_Cluster', 
    n_clusters=100
)


coords_dropoff = df[['dropoff_longitude', 'dropoff_latitude']].dropna()
labels_dropoff = pickup_model.predict(coords_dropoff.values)

df['Dropoff_Cluster'] = -1
df.loc[coords_dropoff.index, 'Dropoff_Cluster'] = labels_dropoff
df['Dropoff_Cluster'] = df['Dropoff_Cluster'].astype(int)

print("Clustering Done!")

# GEO API 

from geopy.exc import GeocoderTimedOut, GeocoderUnavailable


geolocator = Nominatim(user_agent="nyc_taxi_project_student_final_v3", timeout=5)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1.5)

cluster_centers = pickup_model.cluster_centers_ 
cluster_names = {}

for cluster_id, coords in enumerate(cluster_centers):
    lon, lat = coords 
    
    
    lat = float(lat)
    lon = float(lon)

    try:
        location = geocode((lat, lon), language='en')
        
        if location:
            address = location.raw.get('address', {})
            neigh = address.get('neighbourhood', address.get('hamlet', address.get('quarter', 'Unknown')))
            borough = address.get('suburb', address.get('city_district', address.get('borough', 'Unknown')))
            cluster_names[cluster_id] = {'Neighbourhood': neigh, 'Borough': borough}
        else:
            cluster_names[cluster_id] = {'Neighbourhood': 'Unknown', 'Borough': 'Unknown'}
            
    except (GeocoderTimedOut, GeocoderUnavailable):
    
        
        cluster_names[cluster_id] = {'Neighbourhood': 'Error', 'Borough': 'Error'}
    except Exception as e:
        cluster_names[cluster_id] = {'Neighbourhood': 'Error', 'Borough': 'Error'}
        
    if cluster_id % 10 == 0: 
        print(f"  - Zone {cluster_id}/100 traitée...")
        continue


print("Fusion des noms dans le DataFrame...")
df_zones = pd.DataFrame.from_dict(cluster_names, orient='index').reset_index()
df_zones.columns = ['Cluster_ID', 'Neighbourhood', 'Borough']

#Now adjusting pickup and dropoff columns using actual locations based on coords
df = df.merge(
    df_zones.rename(columns={'Cluster_ID': 'Pickup_Cluster', 'Neighbourhood': 'Pickup_Neighbourhood', 'Borough': 'Pickup_Borough'}),
    on='Pickup_Cluster', how='left'
)

df = df.merge(
    df_zones.rename(columns={'Cluster_ID': 'Dropoff_Cluster', 'Neighbourhood': 'Dropoff_Neighbourhood', 'Borough': 'Dropoff_Borough'}),
    on='Dropoff_Cluster', how='left'
)
print("Done")

Clustering de 'Pickup_Cluster' en 100 zones...
Clustering Done!
  - Zone 0/100 traitée...
  - Zone 10/100 traitée...
  - Zone 20/100 traitée...
  - Zone 30/100 traitée...
  - Zone 40/100 traitée...
  - Zone 50/100 traitée...
  - Zone 60/100 traitée...
  - Zone 70/100 traitée...
  - Zone 80/100 traitée...
  - Zone 90/100 traitée...
Fusion des noms dans le DataFrame...
Done


In [54]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,...,Moments_of_day,duration_in_min,Duration_Category,avg_speed,Pickup_Cluster,Dropoff_Cluster,Pickup_Neighbourhood,Pickup_Borough,Dropoff_Neighbourhood,Dropoff_Borough
0,2,2015-01-15 19:05:39,2015-01-15 19:23:42,1,1.59,-73.993896,40.750111,1,0,-73.974785,...,rush hour,18.050000,Medium (10-20m),5.285319,30,36,Chelsea District,Manhattan,Tudor City,Manhattan
1,1,2015-01-10 20:33:38,2015-01-10 20:53:28,1,3.30,-74.001648,40.724243,1,0,-73.994415,...,rush hour,19.833333,Medium (10-20m),9.983193,22,34,University Village,Manhattan,Hudson Yards,Manhattan
2,1,2015-01-10 20:33:38,2015-01-10 20:43:41,1,1.80,-73.963341,40.802788,1,0,-73.951820,...,rush hour,10.050000,Medium (10-20m),10.746269,65,41,Manhattan Community Board 9,Manhattan,Manhattanville,Manhattan
3,1,2015-01-10 20:33:39,2015-01-10 20:35:31,1,0.50,-74.009087,40.713818,1,0,-74.004326,...,rush hour,1.866667,Very Short (<5m),16.071429,46,57,Tribeca,Manhattan,Civic Center,Manhattan
4,1,2015-01-10 20:33:39,2015-01-10 20:52:58,1,3.00,-73.971176,40.762428,1,0,-74.004181,...,rush hour,19.316667,Medium (10-20m),9.318378,93,92,Midtown East,Manhattan,Chelsea District,Manhattan


In [55]:
ordered_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df['day_of_week'] = pd.Categorical(
    df['day_of_week'], 
    categories=ordered_days, 
    ordered=True
)

df['day_of_week'] = df['day_of_week'].cat.codes

In [56]:
# Make moments of day into numerical 
#dic_moments_of_day = { 'early morning' : 0, "morning" : 1, 'noon/afternoon': 2, 'rush hour':3 ,'night': 4}
#df['Moments_of_day'] = df['Moments_of_day'].map(dic_moments_of_day)

moments_of_day = [ 'early morning', "morning", 'noon/afternoon', 'rush hour','night']
df['Moments_of_day'] = pd.Categorical(
    df['Moments_of_day'], 
    categories=moments_of_day, 
    ordered=True
)

df['Moments_of_day'] = df['Moments_of_day'].cat.codes

In [66]:
# Columns to drop
# Practically constant feature : mta_tax
# Unuseful information on its own: datetime type columns and coordinates and tips + total_fare as we are prediction the fare amount and not the total fare amount
# Features that don't really have an impact on the fare : VendorID
columns_to_drop_rf = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'mta_tax', "pickup_longitude", 'pickup_latitude', "dropoff_longitude", 'dropoff_latitude', 
                      'tip_amount', 'total_amount', 'pickup_date', 'duration_in_min', "avg_speed", "Duration_Category", "VendorID", "Pickup_Neighbourhood","Dropoff_Neighbourhood","Pickup_Borough","Dropoff_Borough"]
extra_cols_to_drop = ['RateCodeID', 'store_and_fwd_flag', 'extra', 'tolls_amount','improvement_surcharge']


In [71]:
#Drop the unneeded colulmns
df_rf = df.drop(columns=columns_to_drop_rf)
df_rf.head()

,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,payment_type,fare_amount,extra,tolls_amount,improvement_surcharge,day_of_week,Moments_of_day,Pickup_Cluster,Dropoff_Cluster,predicted_duration
0,1,1.59,1,0,1,12.0,1.0,0.0,0.3,3,3,30,36,12.407930
1,1,3.30,1,0,1,14.5,0.5,0.0,0.3,5,3,22,34,15.619350
2,1,1.80,1,0,2,9.5,0.5,0.0,0.3,5,3,65,41,7.488003
3,1,0.50,1,0,2,3.5,0.5,0.0,0.3,5,3,46,57,7.001586
4,1,3.00,1,0,2,15.0,0.5,0.0,0.3,5,3,93,92,18.895750


In [73]:
df_rf = df_rf.drop(columns=extra_cols_to_drop)
df_rf.head()

,passenger_count,trip_distance,payment_type,fare_amount,day_of_week,Moments_of_day,Pickup_Cluster,Dropoff_Cluster,predicted_duration
0,1,1.59,1,12.0,3,3,30,36,12.407930
1,1,3.30,1,14.5,5,3,22,34,15.619350
2,1,1.80,2,9.5,5,3,65,41,7.488003
3,1,0.50,2,3.5,5,3,46,57,7.001586
4,1,3.00,2,15.0,5,3,93,92,18.895750


In [ ]:


# 1. Définition X et y
X = df_rf.drop(labels='fare_amount', axis=1)
y = df_rf['fare_amount']

# 2. Identification des indices des colonnes catégorielles
# CatBoost a besoin de savoir OÙ sont les colonnes de texte
# Ici, on récupère les noms des colonnes qui sont de type 'object' ou 'category'
cat_features_names = X.select_dtypes(include=['object', 'category']).columns.tolist()

# 3. Train test split
# On n'a pas besoin d'Imputer ni de One-Hot Encoding !
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# 4. Création du modèle
# verbose=0 pour qu'il ne parle pas trop (il est très bavard sinon)
cb = CatBoostRegressor(iterations=500, learning_rate=0.1, depth=6, verbose=0)

# 5. Entraînement
# C'est ici la magie : on lui donne la liste des colonnes catégorielles
cb.fit(X_train, y_train, cat_features=cat_features_names)

# 6. Prédiction & Évaluation
y_pred = cb.predict(X_test)

rmse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'rmse : {rmse}')
print(f'mae  : {mae}')
print(f'r2   : {r2}')

In [60]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- 0. DÉFINITION ET NETTOYAGE DES CATÉGORIES (INDISPENSABLE) ---
# On liste les vraies catégories
cat_features = [
    'VendorID', 'RateCodeID', 'store_and_fwd_flag', 'payment_type',
    'day_of_week', 'Moments_of_day', 'Pickup_Cluster', 'Dropoff_Cluster'
]


# --- ÉTAPE 1 : PRÉDIRE LA DURÉE ---

# 1. X pour la durée ne contient QUE les catégories
X_duration = df_rf[cat_features]
y_duration = df_rf['duration_in_min']

# 2. Entraînement Durée
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_duration, y_duration, test_size=0.8, random_state=42)

cb_duration = CatBoostRegressor(iterations=500, depth=6, loss_function='RMSE', verbose=0)

print("Entraînement modèle Durée...")
cb_duration.fit(X_train_d, y_train_d, cat_features=cat_features)

# 1. Pour l'évaluation (Scores) : On utilise SEULEMENT le Test Set
y_pred_test = cb_duration.predict(X_test_d)

rmse_val = mean_squared_error(y_test_d, y_pred_test)
mae_val = mean_absolute_error(y_test_d, y_pred_test)
r2_val = r2_score(y_test_d, y_pred_test)

print("Pred Temps")
print("-" * 30)
print(f'RMSE : {rmse_val:.4f}')
print(f'MAE  : {mae_val:.4f}')
print(f'R²   : {r2_val:.4f}')
print("-" * 30)

df_rf['predicted_duration'] = cb_duration.predict(X_duration)

# --- ÉTAPE 2 : PRÉDIRE LE PRIX (Chainage) ---

# 1. On définit les features du prix : Catégories + Durée Prédite
# Note : on sépare bien la liste des colonnes totales de la liste des catégories
all_features_price = cat_features + ['predicted_duration']

X_price = df_rf[all_features_price]
y_price = df_rf['fare_amount']

# 2. Entraînement Prix
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_price, y_price, test_size=0.8, random_state=42)

cb_price = CatBoostRegressor(iterations=500, depth=6, loss_function='RMSE', verbose=0)

print("Entraînement modèle Prix...")

# CORRECTION ICI : On utilise 'cat_features' (la liste des catégories pures)
# On ne met PAS 'predicted_duration' dedans, car c'est un chiffre continu (float)
cb_price.fit(X_train_p, y_train_p, cat_features=cat_features) 

# --- 3. ÉVALUATION ---
print("\nCalcul des scores finaux...")
y_pred = cb_price.predict(X_test_p)

rmse = mean_squared_error(y_test_p, y_pred)
mae = mean_absolute_error(y_test_p, y_pred)
r2 = r2_score(y_test_p, y_pred)

print("-" * 30)
print(f'RMSE : {rmse:.4f} $')
print(f'MAE  : {mae:.4f} $')
print(f'R²   : {r2:.4f}')
print("-" * 30)

Entraînement modèle Durée...
Pred Temps
------------------------------
RMSE : 24.4631
MAE  : 3.1745
R²   : 0.7057
------------------------------
Entraînement modèle Prix...

Calcul des scores finaux...
------------------------------
RMSE : 13.9268 $
MAE  : 1.8467 $
R²   : 0.8550
------------------------------


In [64]:
df_rf.head()

,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,payment_type,fare_amount,extra,tolls_amount,improvement_surcharge,day_of_week,Moments_of_day,Pickup_Cluster,Dropoff_Cluster
0,1,1.59,1,0,1,12.0,1.0,0.0,0.3,3,3,30,36
1,1,3.30,1,0,1,14.5,0.5,0.0,0.3,5,3,22,34
2,1,1.80,1,0,2,9.5,0.5,0.0,0.3,5,3,65,41
3,1,0.50,1,0,2,3.5,0.5,0.0,0.3,5,3,46,57
4,1,3.00,1,0,2,15.0,0.5,0.0,0.3,5,3,93,92


In [62]:
#EXPORT


# 1. Sauvegarder le K-Means (pour trouver les clusters Pickup/Dropoff)
# 'pickup_model' est votre MiniBatchKMeans entraîné
joblib.dump(pickup_model, 'kmeans_clusters.pkl')

# 2. Sauvegarder les modèles CatBoost

cb.save_model("model2.cbm")
cb_duration.save_model("model_duration2.cbm")
cb_price.save_model("model_price2.cbm")

#print("Fichiers sauvegardés : kmeans_clusters.pkl, model_duration.cbm, model_price.cbm")